In [1]:
import json
import os
from pathlib import Path

import requests
import yaml
from PIL import Image
from tqdm import tqdm

from utils import make_dirs

In [2]:
def voc_to_yolo(Imgsize, box):
    width, height = Imgsize
    xmin, ymin, xmax, ymax = box
    dw = max(xmin, xmax) - min(xmin, xmax)  # 框的宽
    dh = max(ymin, ymax) - min(ymin, ymax)  # 框的高
    x_ = (xmin + xmax) / 2.0  # 框的中心点x坐标
    y_ = (ymin + ymax) / 2.0  # 框的中心点y坐标

    # 归一化
    cx = round(x_ / width, 6)
    cy = round(y_ / height, 6)
    w = round(dw / width, 6)
    h = round(dh / height, 6)

    return [cx, cy, w, h]


In [3]:
def convert(file, zip=True):
    """Converts Labelbox JSON labels to YOLO format and saves them, with optional zipping."""
    names = []  # class names
    file = Path(file)
    save_dir = make_dirs(file.stem)
    with open(file) as f:
        data = json.load(f)  # load JSON
    data = data["annotations"]
    for img in tqdm(data, desc=f"Converting {file}"):
        im_path = img["filename"]
        im_path = os.path.normpath(im_path).replace('\\', '/')
        images_name = Path(im_path).stem
        im = Image.open(requests.get(im_path, stream=True).raw if im_path.startswith("http") else im_path)  # open
        width, height = im.size  # image size
        label_path = save_dir / "labels" / Path(images_name).with_suffix(".txt").name
        image_path = save_dir / "images" / Path(images_name).with_suffix(".jpg").name
        im.save(image_path, quality=95, subsampling=0)

        
        box = img["box"]
        if (None in box.values()):
            continue
        xywh = voc_to_yolo(im.size,box.values())

        # class
        cls = img["label"]  # class name
        if cls not in names:
            names.append(cls)

        line = names.index(cls), *xywh  # YOLO format (class_index, xywh)
        with open(label_path, "a") as f:
            f.write(("%g " * len(line)).rstrip() % line + "\n")

    # Save dataset.yaml
    d = {
        "path": f"../datasets/{file.stem}  # dataset root dir",
        "train": "images/train  # train images (relative to path) 1280*720 images",
        "val": "images/val  # val images (relative to path) 1280*720 images",
        "test": " # test images (optional)",
        "nc": len(names),
        "names": names,
    }  # dictionary

    with open(save_dir / file.with_suffix(".yaml").name, "w") as f:
        yaml.dump(d, f, sort_keys=False)

    # Zip
    if zip:
        print(f"Zipping as {save_dir}.zip...")
        os.system(f"zip -qr {save_dir}.zip {save_dir}")

    print("Conversion completed successfully!")

In [4]:
convert("train.json")

Converting train.json: 100%|██████████| 42/42 [00:00<00:00, 74.23it/s]

Zipping as train.zip...
Conversion completed successfully!
